# Part 1: Imports, data uploads and preparation.

As is customary, let us first call the Python libraries needed here, and upload the needed data and code.

In [ ]:
from model import setup, balance_calcs, func_FDC, visuals, performance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Loading model and water balance for historical data

In this tutorial we will compute performance, then compare it for the scenarios defined in Tutorial 3. First let's compute the historical water balance.

In [ ]:
# Preparing the model
reservoir_name = 'Conowingo'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Baltimore', 'Chester', 'Nuclear plant']

# Loading the model!
conowingo = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Read flow and demand data. We keep this copy of the data for the simulation of different futures.
flows_default = setup.extract_flows(reservoir=conowingo)
display(flows_default)

In [ ]:
# First, make a copy of the flows to initialise the water balance
historical_balance = flows_default.copy()  # Keep flows_default as an untouched copy

# Computing the water balance for our standard operating policy (SOP)
balance_calcs.sop_full(reservoir=conowingo, water_flows=historical_balance)

# Part 2: performance metrics

Management objectives are as follows:
1) Produce hydropower
2) Meet environmental flows
3) Meet domestic and industrial demands
4) Avoid excessive flooding that would require evacuating the downstream town of “Port Deposit” (15,000 m3/s)
5) Maintain a water level compatible with recreation (hydraulic head over 106.5 ft, where 1ft = 0.3048 m) in June, July and August.

First, we will explore objective (1), and we will then focus on the other objectives. For these objectives, we will compare the water flows / levels versus a threshold, and use the R-R-V indicators defined in the lecture (reliability / resilience / vulnerability).


## 2.1 - Hydropower

For Conowingo Dam, key parameters of the hydropower plant can be found in the **"Reservoir characteristics"** part of the spreadsheet.

In [ ]:
# First we download the key data
key_parameters = pd.read_excel('data/Conowingo_data.xlsx', sheet_name='Reservoir characteristics')

# We define the hydropower station parameters
installed_capacity = key_parameters.iloc[4, 1]
nominal_head = key_parameters.iloc[3, 1]
max_release = key_parameters.iloc[5, 1]

# Variables
rho = 1000 # Density of water, kg/m3
g = 9.81 # Acceleration of gravity, m/s2

From this we can use the equation given in the lecture, linking hydropower station parameters to power production, to deduce the combined efficiency of converting potential energy into kinetic energy (through the turbine) and of converting this kinetic energy to electricity.

In [ ]:
# Deduce efficiency
efficiency = installed_capacity*1E6 / (rho*g*nominal_head*max_release)
print("Turbines and plant combined efficiency is " + "{:.3f}".format(efficiency))

With these parameters and the water balance, we can get time series of daily hydropower production.

In [ ]:
# Get hydropower production time series

n_steps = len(historical_balance)

# Get release time series (capped by max release)
release = np.minimum(historical_balance['Outflows (m3/s)'].values, np.ones(n_steps)*max_release)

# Get hydraulic head time series
hy_head = np.zeros(n_steps)
for t in range(n_steps):
  depth = conowingo.get_height(historical_balance.iloc[t, -1])
  hy_head[t] = nominal_head - conowingo.total_lake_depth + depth

# Deduce daily hydropower production time series (in MWh)
hydropower_daily = pd.Series(index=historical_balance.index, data=rho*g*efficiency*np.multiply(hy_head, release)*24/1E6, name='Daily hydropower production (MWh)')

plt.plot(hydropower_daily.index, hydropower_daily/1000)
plt.xlabel('Year', size=14)
plt.ylabel('Daily hydropower production (GWh)', size=14)

This is not a very helpful visual, and aggregation to annual data will certainly be helpful.

In [ ]:
# Let us get annual hydropower production

# First the headline number: average over 70 years (GWh)
print('Annual average hydropower production at Conowingo is ' + "{:.0f}".format(hydropower_daily.sum() / 70 / 1000) + ' GWh')

In [ ]:
# Now the time series
hydropower_annual = hydropower_daily.resample('YE').sum()/1000
hydropower_annual.name = 'Annual hydropower production (GWh)'

print('Check we have the same average. \nAnnual average hydropower production at Conowingo is ' + "{:.0f}".format(hydropower_annual.mean()) + ' GWh')

In [ ]:
# Let us plot it
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(hydropower_annual.index.year, hydropower_annual, c='b', label='No intake')
ax.set_xlabel('Date', size=14)
ax.set_ylabel('Annual power production (GWh)', size=14)

# We set the boundaries of the x-axis
ax.set_xlim(hydropower_annual.index.year[0], hydropower_annual.index.year[-1])

In [ ]:
# let us compare with annual average flows.
fig = visuals.annual_average(daily_data=pd.Series(historical_balance['Total inflows (m3/s)']) * 86400/1E9, 
                             data_label='total inflow ($km^3$)')

**Question 1. What can we say about the correlation between inflows and hydropower production?**


## 2.2 - R-R-V indicators

Let us now introduce a function to compute the R-R-V performance metrics introduced in the lecture.

**Question 2. In the function below, do you recognise the formulas given in the lecture? What are extra steps that need to be taken? And why do we measure vulnerability as percentage?**

In [ ]:
def rrv_indicators(time_series, dynamic_threshold, above_desirable, name, **kwargs):
    """
    Compute the RRV indicators for a time series vs. a threshold. Arguments:
        time_series: numpy vector
        dynamic_threshold: numpy vectors of same length as `time_series`
        above_desirable: boolean. If True we value staying at or above a threshold.
        name: String, the name of the site
        optional argument `vul_unit`: String, default as a percentage, to specify how vulnerability is evaluated
    Returns a pandas DataFrame with several perf_metrics metrics.
    """

    # Optional argument
    vul_unit = kwargs.pop("vul_unit", '%')

    # Local variables
    n_steps = len(time_series)
    tolerance = 1E-6  # for rounding errors

    # If above_desirable is false we need to change sign of all data now, so we compare a and b
    a = (2 * above_desirable - 1) * time_series
    b = (2 * above_desirable - 1) * dynamic_threshold
    b = b - tolerance

    # Initialise output
    indicators = pd.DataFrame(columns=['Name', 'Reliability (0-1)', 'Resilience (-)', 'Vulnerability', 'Failure count'])
    indicators.loc[0, 'Name'] = name

    # Reliability
    indicators.loc[0, 'Reliability (0-1)'] = 1 - np.sum(a < b) / n_steps

    # We need to count failure events to compute resilience and vulnerability
    event_count = 0
    # We also need to have the maximal amplitude or magnitude of failure
    magnitude = []
    # We use a while loop to count events and their magnitude
    t = 0
    while t < n_steps:

        if a[t] < b[t]:
            # New event! we need to update the count of failure events
            event_count = event_count + 1
            # We also need to keep track of the maximum amplitude of failure
            # By default failure is expressed in relative terms
            if vul_unit == '%':
                magnitude.append((b[t] - a[t]) / abs(b[t]))
            else:
                magnitude.append(b[t] - a[t])
            # Now while event lasts
            while a[t] < b[t]:
                t = t+1
                if t == n_steps:
                    break
                if vul_unit == '%':
                    magnitude[-1] = max(magnitude[-1], (b[t] - a[t]) / abs(b[t]))
                else:
                    magnitude[-1] = max(magnitude[-1], b[t] - a[t])

        # Time increment so while loop concludes
        t = t+1

    # Resilience
    indicators.loc[0, 'Resilience (-)'] = event_count / (n_steps * (1 - indicators.loc[0, 'Reliability (0-1)']))

    # Vulnerability (as a percentage)
    if vul_unit == '%':
        indicators.loc[0, 'Vulnerability'] = "{:.0f}".format(np.mean(magnitude) * 100) + '%'
    else:
        indicators.loc[0, 'Vulnerability'] = "{:.2f}".format(np.mean(magnitude)) + vul_unit

    # Finally, exporting the failure count
    indicators.loc[0, 'Failure count'] = event_count

    return indicators

In [ ]:
# Application to all four Conowingo demands
metrics = pd.concat([rrv_indicators(historical_balance['Withdrawals Baltimore (m3/s)'].to_numpy(), 
                                    historical_balance['Baltimore demand (m3/s)'].to_numpy(), True, 'Baltimore'),
                     rrv_indicators(historical_balance['Withdrawals Chester (m3/s)'].to_numpy(), 
                                    historical_balance['Chester demand (m3/s)'].to_numpy(), True, 'Chester'),
                     rrv_indicators(historical_balance['Withdrawals Nuclear plant (m3/s)'].to_numpy(), 
                                    historical_balance['Nuclear plant demand (m3/s)'].to_numpy(), True, 'Nuclear'),
                     rrv_indicators(historical_balance['Outflows (m3/s)'].to_numpy(), 
                                    historical_balance['Environmental demand (m3/s)'].to_numpy(), True, 'Env. flows')],
                     axis=0, ignore_index=True)

print('Performance metrics for demands are:\n')
display(metrics)

In [ ]:
# Same for flooding
flooding_metrics = rrv_indicators(time_series=historical_balance['Outflows (m3/s)'].to_numpy(), 
                                  dynamic_threshold=15000*np.ones(len(historical_balance)), 
                                  above_desirable=False, 
                                  name='Flooding')

metrics = pd.concat([metrics, flooding_metrics], axis=0, ignore_index=True)

print('Performance metrics including demands and flooding are:\n')
display(metrics)

**Question 3. Compute how long it takes a flood at the flooding threshold to completely fill Conowingo, and use this to comment on the reliability, resilience and vulnerability for the flooding objective. In particular, can we use different operating policies to avoid flooding the town downstream of Conowingo (Port Deposit)?**

We have now computed metrics for objectives (1) to (4) under historical flows and the standard operating policy (SOP). We now need to evaluate the last objective, summer recreation. The key difference is that the three over seasons (75% of time) don't count towards reliability calculations. This necessitates extra leg work.

In [ ]:
# Summer recreation (lake levels need to stay above a certain level in June, July and August)

# We need time series of level objectives. We initialise at 0 requirement.
level_objective = pd.Series(index=historical_balance.index, data=np.zeros(len(historical_balance)))

# We set a level during summer months, to be compared with lake level (which coincide with hydraulic head)
summer_requirement = 106.5*0.3048
for month in np.arange(6, 9, 1):
    level_objective[level_objective.index.month == month] = summer_requirement

# Get hydraulic head time series, assuming linear relationship between depth and lake area
hydraulic_head = np.zeros(len(historical_balance))
for t in range(len(historical_balance)):
    depth = conowingo.get_height(historical_balance.iloc[t, -1])
    hydraulic_head[t] = conowingo.hydropower_plant.nominal_head - conowingo.total_lake_depth + depth

# Get the indicators
recreation_metrics = rrv_indicators(hydraulic_head, level_objective.to_numpy(), True, 'Recreation', vul_unit='m')

# We need to account for the fact that this requirement is for three months only, which impacts reliability
# Failure happens more often if measured in the shorter time window
recreation_metrics.iloc[0, 1] = 1 - (1-recreation_metrics.iloc[0, 1]) * len(level_objective) / (70*(30+31+31))

metrics = pd.concat([metrics, recreation_metrics], axis=0, ignore_index=True)
print('Performance metrics including demands, flooding and recreation are:\n')
display(metrics)

In [ ]:
# Add a new column, volumetric reliability
metrics.insert(5, 'Volumetric reliability', [0, 0, 0, 0, 'N/A', 'N/A'])
display(metrics)

In [ ]:
# Volumetric reliability is only defined for the demands, and it relies on the grand total supply / demand
totals = historical_balance.sum(axis=0)

metrics.loc[0, 'Volumetric reliability'] = totals['Withdrawals Baltimore (m3/s)'] / totals['Baltimore demand (m3/s)']
metrics.loc[1, 'Volumetric reliability'] = totals['Withdrawals Chester (m3/s)'] / totals['Chester demand (m3/s)']
metrics.loc[2, 'Volumetric reliability'] = totals['Withdrawals Nuclear plant (m3/s)'] / totals['Nuclear plant demand (m3/s)']
metrics.loc[3, 'Volumetric reliability'] = np.sum(np.minimum(historical_balance['Environmental demand (m3/s)'], 
                                                             historical_balance['Outflows (m3/s)'])) / totals['Environmental demand (m3/s)']

display(metrics)

**Question 4. Which objectives do you feel the chosen operating policy favours? Why?**

# Part 3: performance change with climate change

Let us now examine how the climate changes modelled in Tutorial 3 might affect performance, compared with the historical balance. Recall the two models for streamflow change:
1) Uniform decrease affecting all flows equally. As in tutorial 3 we assume a 20% decrease in flows.
2) A model combining a 20% average decrease in inflows, with a 50% increase in variability and a 40% decrease in the first percentile of flows (low flows). In other words, the model represents an increase in both high and low flows.

## 3.1 - Recall what these models look like

In [ ]:
# Water balance for the uniform 20% decrease
uniform_drier = balance_calcs.uniform_change_model(flows_default, 0.8)  # Initialise water balance
balance_calcs.sop_full(conowingo, uniform_drier)

In [ ]:
# Water balance for the 20% average decrease with amplified extremes
mean_multiplier = 0.8
variability_multiplier =1.5
low_flow_multiplier = 0.6
drier_more_extreme = balance_calcs.amplified_extremes_model(flows_default, [mean_multiplier, variability_multiplier, low_flow_multiplier], 1) 
balance_calcs.sop_full(conowingo, drier_more_extreme)

Remember the difference between the models!

In [ ]:
fig = visuals.compare_fdc(reference=pd.Series(historical_balance['Total inflows (m3/s)']), 
                          alternative=pd.Series(uniform_drier['Total inflows (m3/s)']), 
                          alternative_2=pd.Series(drier_more_extreme['Total inflows (m3/s)']), 
                          labels=['Historical', 'Drier future (uniform)', 'Drier and more extreme future'])

And remember what that means for storage during a dry period. **What could that mean for performance?**

In [ ]:
fig = visuals.compare_storage_timeseries(reservoir=conowingo, 
                                         storage_1=pd.Series(historical_balance['Storage (m3)']),
                                         storage_2=pd.Series(uniform_drier['Storage (m3)']), 
                                         storage_3=pd.Series(drier_more_extreme['Storage (m3)']),
                                         labels=['Historical', 'Drier future (uniform)', 'Drier and more extreme future'],
                                         first_date=datetime.date(1962, 1, 1), 
                                         last_date=datetime.date(1970, 1, 1))

## 3.2 - Performance comparison

All the performance code is also available in `model/performance.py` which can be imported using `from model import performance`.

We display one above the other the full metrics for:
1) Historical conditions.
2) Model 1, uniform 20% inflow decrease.
3) Model 2, 20% average inflow decrease with extreme amplification.

In [ ]:
display(metrics)  # 1 - Historical conditions

In [ ]:
metrics_1 = performance.all_metrics(conowingo, uniform_drier)
display(metrics_1)  # 2 - Model 1, uniform 20% inflow decrease.

In [ ]:
metrics_2 = performance.all_metrics(conowingo, drier_more_extreme)
display(metrics_2)  # 3 - Model 2, 20% average inflow decrease with extreme amplification.

**Question 5. How did the choice of model affect performance?**

**Question 6. Try changing parameters of statistical model. For instance, what happens to performance if instead of a 20% decrease, the mean were unchanged compared with historical conditions?**